In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/archive.zip" -d /content/


Streaming output truncated to the last 5000 lines.
  inflating: /content/Training/glioma/Tr-gl_0712.jpg  
  inflating: /content/Training/glioma/Tr-gl_0713.jpg  
  inflating: /content/Training/glioma/Tr-gl_0714.jpg  
  inflating: /content/Training/glioma/Tr-gl_0715.jpg  
  inflating: /content/Training/glioma/Tr-gl_0716.jpg  
  inflating: /content/Training/glioma/Tr-gl_0717.jpg  
  inflating: /content/Training/glioma/Tr-gl_0718.jpg  
  inflating: /content/Training/glioma/Tr-gl_0719.jpg  
  inflating: /content/Training/glioma/Tr-gl_0720.jpg  
  inflating: /content/Training/glioma/Tr-gl_0721.jpg  
  inflating: /content/Training/glioma/Tr-gl_0722.jpg  
  inflating: /content/Training/glioma/Tr-gl_0723.jpg  
  inflating: /content/Training/glioma/Tr-gl_0724.jpg  
  inflating: /content/Training/glioma/Tr-gl_0725.jpg  
  inflating: /content/Training/glioma/Tr-gl_0726.jpg  
  inflating: /content/Training/glioma/Tr-gl_0727.jpg  
  inflating: /content/Training/glioma/Tr-gl_0728.jpg  
  inflating: /

In [3]:
!ls /content/


drive  sample_data  Testing  Training


In [4]:
!ls /content/Training


glioma	meningioma  notumor  pituitary


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "/content/Training",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    "/content/Testing",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [7]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


In [8]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [11]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=15
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 123s 579ms/step - accuracy: 0.7103 - loss: 0.7279 - val_accuracy: 0.8169 - val_loss: 0.4597
Epoch 2/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 77s 428ms/step - accuracy: 0.8699 - loss: 0.3419 - val_accuracy: 0.8658 - val_loss: 0.3597
Epoch 3/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 73s 409ms/step - accuracy: 0.8879 - loss: 0.2923 - val_accuracy: 0.8337 - val_loss: 0.4376
Epoch 4/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 75s 417ms/step - accuracy: 0.9073 - loss: 0.2508 - val_accuracy: 0.8810 - val_loss: 0.3035
Epoch 5/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 74s 415ms/step - accuracy: 0.9136 - loss: 0.2184 - val_accuracy: 0.8665 - val_loss: 0.3386
Epoch 6/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 76s 424ms/step - accuracy: 0.9154 - loss: 0.2178 - val_accuracy: 0.8894 - val_loss: 0.2836
Epoch 7/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 75s 421ms/step - accuracy: 0.9250 - loss: 0.1985 - val_accuracy: 0.9024 - val_loss: 0.2618
Epoch 8/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 75s 420ms/step - accuracy: 0.9327 - loss: 

In [12]:
loss, accuracy = model.evaluate(test_data)
print("Test Accuracy:", accuracy*100)


41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.9144 - loss: 0.2134
Test Accuracy: 90.84668159484863


In [13]:
model.save("/content/brain_tumor_model.h5")


In [14]:
from google.colab import files
files.download("/content/brain_tumor_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train_data, validation_data=test_data, epochs=5)


Epoch 1/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 172s 648ms/step - accuracy: 0.5937 - loss: 2.4040 - val_accuracy: 0.7818 - val_loss: 0.8319
Epoch 2/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 80s 449ms/step - accuracy: 0.8439 - loss: 0.4689 - val_accuracy: 0.8124 - val_loss: 0.6491
Epoch 3/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 80s 448ms/step - accuracy: 0.8754 - loss: 0.3441 - val_accuracy: 0.8284 - val_loss: 0.5744
Epoch 4/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 81s 454ms/step - accuracy: 0.8983 - loss: 0.2827 - val_accuracy: 0.8474 - val_loss: 0.5020
Epoch 5/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 82s 456ms/step - accuracy: 0.9147 - loss: 0.2234 - val_accuracy: 0.8429 - val_loss: 0.5001


In [16]:
!pip install gradio


In [17]:
from google.colab import files
files.upload()


Saving background.jpg to background.jpg


{'background.jpg': b'\xff\xd8\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a((((((((((((((((((((((((((((((((((((((((((((((((((\xff\xc0\x00\x11\x08\x02\x1c\x03\xc0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x01\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x01\x03\x04\x05\x06\x07\x08\xff\xc4\x00F\x10\x00\x01\x03\x03\x03\x02\x04\x02\x07\x07\x03\x03\x02\x04\x07\x00\x01\x00\x02\x03\x04\x05\x11\x12!1AQ\x06\x13"a2q\x14#BR\x81\x91\xa1\x153br\xb1\xc1\xd1\x07$CS\x82\x924\xe1\x16%cs5D\x83\x93\xc2\xf0\xf1\xff\xc4\x00\x1b\x01\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x03\x00\x01\x04\x05\x06\x07\xff\xc4\x005\x11\x00\x02\x02\x02\x02\x01\x03\x02\x03\x07\x04\x02\x03\x01\x00\x00\x00\x01\x02\x03\x04\x11\x12!1\x05\x13A"Q\x142a#Bq\

In [18]:
import tensorflow as tf

model = tf.keras.models.load_model("/content/brain_tumor_model.h5")


In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image

class_names = list(train_data.class_indices.keys())

def predict_brain_tumor(img):
    img = img.resize((224,224))
    img_array = np.array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    return f"Prediction: {predicted_class}\nConfidence: {confidence:.2f}%"


In [23]:
import base64

with open("/content/background.jpg", "rb") as f:
    img_bytes = f.read()

encoded_bg = base64.b64encode(img_bytes).decode("utf-8")



In [24]:
bg_css = f"""
.gradio-container {{
    background-image: url("data:image/jpeg;base64,{encoded_bg}");
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
}}

.gradio-container::before {{
    content: "";
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: rgba(0,0,0,0.55);
    z-index: -1;
}}
"""


In [25]:
import gradio as gr

with gr.Blocks(css=bg_css) as demo:

    gr.Markdown(
        "<h1 style='text-align:center; color:white;'>🧠 Brain Tumor Detection System</h1>"
    )

    gr.Markdown(
        "<p style='text-align:center; color:white;'>Upload MRI Brain Scan to Detect Tumor Using Deep Learning</p>"
    )

    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload MRI Brain Scan")
        result_output = gr.Textbox(label="🧾 Diagnosis Result")

    predict_btn = gr.Button("🔍 Predict")

    predict_btn.click(
        fn=predict_brain_tumor,
        inputs=image_input,
        outputs=result_output
    )

demo.launch(share=True)


/tmp/ipython-input-3572646191.py:3: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=bg_css) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aaca121d5964aa291d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
